# Create Component

# Create Registry

Registries enable sharing of assets like components, environments, and models across multiple teams and workspaces in your organization. See details at https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-registries?tabs=portal

There are three ways to create a registry:
* From Azure CLI.
* From Azure Portal.
* From AzureML Studio.

If you are using Azure CLI, you should define the registry with a YAML file first and you can create a registry using the following command:
`az ml registry create --file registry.yml`

**Make sure you've updated to the latest CLI version to run this CLI command.** To check yourCLI version, run `az --version`. If you are using an older version, please update to the latest version by running `az upgrade`.

## Register Component

Components are reusable building blocks of Machine Learning pipelines in AzureML. Creating a component in a registry allows you to use the component in any pipeline in any workspace within your organization. See details at https://learn.microsoft.com/en-us/azure/machine-learning/how-to-share-models-pipelines-across-workspaces-with-registries?tabs=cli#create-a-component-in-registry

There are two ways to register a component:
* From Azure CLI.
* From Python SDK.

We have provided a sample component in the `component` folder. There is a `train.yml` file defining 
the component and a `train_scr/train.py` file defining the component logic. 
You can register the component using the following command:
`az ml component create --file train.yml --registry-name <registry-name>`